# Tell

## A Quick Introduction to Mixed Integer Linear Programming

*Levi John Wolf* [levi.john.wolf@bristol.ac.uk](mailto:levi.john.wolf@bristol.ac.uk)

[Spatial optimisation](https://doi.org/10.1080/00045608.2012.685044) is the process of solving geographical planning problems using *mathematical programming*. As a starting point, understanding *mathematical programming* is an important step in understanding spatial optimisation methods. Thus, in this workshop, we will start understanding spatial optimisation with a quick tour of an *introductory* mathematical program, the "Knapsack" problem. 

### Vocabulary

Every mathematical programming problem has three main parts. First, we first represent the specific decisions we have to make about what site to pick for a fire station or what land use to choose for a parcel of land. These are represented by *decision variables*, and they reflect the things that we can change in order to bring about our desired result. Second, another set of equations, the *constraints*, are used to express the requirements that our decision variables must have in our ideal solution. A solution is called *feasible* when it satisfies all of the *constraints* in a problem, and is *infeasible* when one or more constraints is violated by a proposed solution. Third, we have to specify an *objective function*, which is a calculation that depends on the decision variables which measures how "good" a solution is.  A feasible solution can be an *optimal solution* when the solution's *objective value* is the highest it can be while obeying the problem constraints. Thus, the goal of mathematical programming is find the best *decision variable* values that maximise the *objective function* while obeying all *constraints*. The role of a *solution technique* is to take the information contained in the constraints and objective function and generate feasible (hopefully optimal) solutions. This workshop will focus *less* on the specifics of solution techniques in order to focus *more* on how to represent problems in spatial optimisation models. 

### The Knapsack Problem

The knapsack problem is a very simple mathematical programming problem that is designed to mimick planning for packing to go on a trip: we would like to make sure that we *pack our backpack in a way that maximizes the benefit of items we bring.*  In our case, we have $i = 1, 2, ..., n$ items, and we are trying to decide which of the $n$ to bring to school today. This *decision variable*, $x_i$, will be used to represent whether we bring an item or not. Thus, it can either take a value of $0$ if we bring the item or $1$ if we do not. Each item has some benefit $b_i$ to us throughout the school day, so our *objective function* is the sum of the benefits for items we bring. Mathematically, we can represent this as the sum of the benefit value times the decision variable for each item: 

$$ b_1 * x_1 + b_2 * x_2 + ... + b_n * x_n = \sum_i^n b_ix_i$$

When we do not take an item ($x=0$), we derive no benefit. But, when we *do* take an item ($x=1$), it contributes $b_i$ benefit. This is the *objective function*, since it measures how much benefit we get from our decisions $x$. We would like to *maximize* this, because we intend to maximize the benefit we get from the items we take. 

However, if we were just able to take all of our items, we would not have a decision problem! Indeed, knapsacks have *limited size*, and we can only fit so many items before it is full. We want to make sure that we only take items that will fit into our knapsack: we want to place a *constraint on the total size of things we take*. To do this, we can imagine that items have a *size*, $s_i$ and the total size of our load is $\sum_i^n s_i*x_i$ (like our benefit equation above). If we want to make sure that the total size of our load is *smaller than* some knapsack size $c$, we specify the following *size constraint*: 

$$ \sum_i^n s_i x_i \leq c $$

Thus, we have just outlined the following *mathematical program*:

<div class="alert alert-success" role="alert">
<b>The Knapsack Problem </b>

*Maximize:* 

$\sum_i^n b_ix_i$

*Subject To:*

1. *Size constraint:*&emsp; $\sum_i^n s_i x_i \leq c$
2. *Binary decisions:*&emsp; $x_i \in {0,1} \ \ \forall \ \ i = 1, 2, ..., n$
</div>

In this problem, we have one size constraint as outlined above, and we have *n* decision variables that are constrained to be either zero or one. Thus, we have $n+1$ constraints in total: the number of decision variables is often added to the number of constraints, because the conditions we place on the decision are themselves ways that the values of the problem must be constrained. 

Solving this problem can be done in a few ways. We will not cover specific solution techniques here, as problems quickly get too complicated to solve using methods that are appropriate for *this* problem. Instead, we will now pivot to *showing* you how to set up and solve the knapsack problem in Python. 

# Show

For now, we will use the following two libraries: 

In [1]:
import pandas # work with tabular data in Python
import pulp # the python package for mathematical programming

In our example, let's say you're packing the following items for school into your backpack, which has a total size of 25 units: 

In [2]:
items = pandas.DataFrame.from_dict(
    dict(
        item_name = ("pencils", "notebook", "extra paper", "pens", "water bottle", "chewing gum", "spare shoes", "lunch", "granola bar"),
        size = (1, 5, 3, 2, 8, 1, 10, 6, 4),
        reward = (2, 2, 1, 1, 3, 1, 2, 2, 1),
    )
).set_index('item_name')

In [3]:
items

,size,reward
item_name,,
pencils,1,2
notebook,5,2
extra paper,3,1
pens,2,1
water bottle,8,3
chewing gum,1,1
spare shoes,10,2
lunch,6,2
granola bar,4,1


Each item (such as pencils or your lunch) has a *size* that records how big the item is, and a *reward* that governs how much benefit you get from taking the item. Remember, our goal is to set up the following linear program: 


<div class="alert alert-success" role="alert">
<b>The Knapsack Problem </b>

*Maximize:* 

$\sum_i^n b_ix_i$

*Subject To:*

1. *Size constraint:*&emsp; $\sum_i^n s_i x_i \leq c$
2. *Binary decisions:*&emsp; $x_i \in {0,1} \ \ \forall \ \ i = 1, 2, ..., n$

</div>


So, to start setting up a linear program, we use the `pulp.LpProblem()` function, which requires us to give the problem a name, and also tell `pulp` whether we intend to *maximize* or *minimize* the objective of this problem. The knapsack is a maximization problem, so we will provide the following: 

In [4]:
knapsack = pulp.LpProblem("knapsack", pulp.LpMaximize)

Then, we can define our decision variables using `pulp.LpVariable()`. This has a few helper methods; we will use `.dict()` to quickly make a *dictionary* (Python's version of a "hashmap" or JSON-like datastructure of key:value pairs) containing all of the decision variables by name: 

In [5]:
take_item_to_school = pulp.LpVariable.dicts("x", items.index, lowBound=0, upBound=1, cat=pulp.LpInteger)

In [6]:
take_item_to_school

{'pencils': x_pencils,
 'notebook': x_notebook,
 'extra paper': x_extra_paper,
 'pens': x_pens,
 'water bottle': x_water_bottle,
 'chewing gum': x_chewing_gum,
 'spare shoes': x_spare_shoes,
 'lunch': x_lunch,
 'granola bar': x_granola_bar}

Then, we can define the "benefit" part of the objective function by multiplying each item's decision variable by its "reward" from our dataframe, and then adding it up: 

In [7]:
benefit = sum( # sum of: 
    items.loc[item_name, 'reward'] * take_item_to_school[item_name] # b_i * x_i
    for item_name in items.index # for each i
)

`pulp` gives you a nice display of the constraints, so that you can check your specification as you go: 

In [8]:
benefit

1*x_chewing_gum + 1*x_extra_paper + 1*x_granola_bar + 2*x_lunch + 2*x_notebook + 2*x_pencils + 1*x_pens + 2*x_spare_shoes + 3*x_water_bottle + 0

To *add* this objective to the `knapsack` problem we set up earlier, we add the statement to the knapsack object: 

In [9]:
knapsack += benefit

Like the benefit statement, we can construct our *capacity* sum in a similar fashion: 

In [10]:
capacity = sum( # sum of: 
    items.loc[item_name, 'size'] * take_item_to_school[item_name] # s_i * x_i
    for item_name in items.index # for each i
) <= 25 # should be no bigger than 25 units

Like before, we can inspect the constraint: 

In [11]:
capacity

1*x_chewing_gum + 3*x_extra_paper + 4*x_granola_bar + 6*x_lunch + 5*x_notebook + 1*x_pencils + 2*x_pens + 10*x_spare_shoes + 8*x_water_bottle + -25 <= 0

Note that the `25` value has been moved from the "right-hand" size of the inequality to the left. `pulp` puts equations into a "canonical form", which we will not cover in full, but it suffices here to note that any values on the right-hand size of a constraint get moved to the left to ensure that a "canonical constraint" always has something $\leq 0$.

We add the constraint to the problem in the same way as the objective: 

In [12]:
knapsack += capacity

You can inspect the model to see how `pulp` represents it internally. Fortunately for us, this should correspond pretty closely to our statement of the **Knapsack Problem** from above:  


In [13]:
knapsack

knapsack:
MAXIMIZE
1*x_chewing_gum + 1*x_extra_paper + 1*x_granola_bar + 2*x_lunch + 2*x_notebook + 2*x_pencils + 1*x_pens + 2*x_spare_shoes + 3*x_water_bottle + 0
SUBJECT TO
_C1: x_chewing_gum + 3 x_extra_paper + 4 x_granola_bar + 6 x_lunch
 + 5 x_notebook + x_pencils + 2 x_pens + 10 x_spare_shoes + 8 x_water_bottle
 <= 25

VARIABLES
0 <= x_chewing_gum <= 1 Integer
0 <= x_extra_paper <= 1 Integer
0 <= x_granola_bar <= 1 Integer
0 <= x_lunch <= 1 Integer
0 <= x_notebook <= 1 Integer
0 <= x_pencils <= 1 Integer
0 <= x_pens <= 1 Integer
0 <= x_spare_shoes <= 1 Integer
0 <= x_water_bottle <= 1 Integer

Now, to solve the model, we use the `.solve()` method of the problem. This will dispatch to an open-source solver by default, and print a significant amout of information from the solution technique. We'll ignore this information for now: 

In [14]:
knapsack.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Apr 19 2023 

command line - cbc /var/folders/g3/kfqgyscs7k59d1l6867ztwmh0000gn/T/126df30d3a14413286faddfb0321f67d-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/g3/kfqgyscs7k59d1l6867ztwmh0000gn/T/126df30d3a14413286faddfb0321f67d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 43 RHS
At line 45 BOUNDS
At line 55 ENDATA
Problem MODEL has 1 rows, 9 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 11.6667 - 0.00 seconds
Cgl0004I processed model has 1 rows, 9 columns (9 integer (9 of which binary)) and 9 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.333333
Cbc0038I Pass   1: suminf.    0.12500 (1) obj. 11.625 iterations 1
Cbc0038I Solution found of 9
Cbc0038I Rounding solution of 10 is better 

1

Note that, by the end of the computation, we see `Result - Optimal solution found`, and the return value is `1`, meaning "success". The results of the computation are stored in the object, and the different variables we built above. So, the decision variables in `take_item_to_school` should record our result:

In [15]:
take_item_to_school['pencils'].varValue

1.0

So, given the size/reward tradeoffs we outlined above in `items`, we should take our pencils to school! You can also see, for example, the objective value: 

In [16]:
knapsack.objective.value()

11.0

To pull these *all* out and assign them into our dataframe, we can use a comprehension like before: 

In [17]:
items.assign(was_taken = [take_item_to_school[item_name].varValue for item_name in items.index])

,size,reward,was_taken
item_name,,,
pencils,1,2,1.0
notebook,5,2,1.0
extra paper,3,1,0.0
pens,2,1,1.0
water bottle,8,3,1.0
chewing gum,1,1,1.0
spare shoes,10,2,0.0
lunch,6,2,1.0
granola bar,4,1,0.0


Thus, we should take our pencils, notebook, pens, waterbottle, chewing gum, and our lunch, which gives us a benefit of 2+2+1+3+1+2 = 11, as we saw before. This means we're taking 1+5+2+8+1+6=23 units of volume, leaving 2 extra units of room. We are leaving behind our granola bar snack, our spare shoes for gym class, and extra paper, since none of those can fit in the remaining 2 units of space. Sometimes, this left-over amount is called the *slack* for that constraint. It is very useful in thinking about solution techniques and formulating additional problems, but we'll ignore it for now. 

Regardless: we have solved our problem 🎉🙌

# Do

Now it is your turn. Using the techniques shown above, can you solve the following *extension* problems? 

## An Important Math Test

You have a very important math test today. In order to be prepared for it, you have to consider [a few different items](https://vimeo.com/77451201) because of this: 

In [18]:
test_items = pandas.DataFrame.from_dict(
    dict(
        item_name = ("protractor", "calculator", "pair of compasses", "garry gum"),
        size = (1, 4, 2, 1),
        reward = (2, 4, 2, 2)
        
    )
).set_index("item_name")

math_test = pandas.concat((items, test_items))
math_test

,size,reward
item_name,,
pencils,1,2
notebook,5,2
extra paper,3,1
pens,2,1
water bottle,8,3
chewing gum,1,1
spare shoes,10,2
lunch,6,2
granola bar,4,1


<div class="alert alert-warning">
Using the methods we discussed above, can you create a new knapsack problem object to solve this <b>new</b> problem? Are you going to be well-prepared for your math test? why or why not?
</div>

## The Grand Gum Ban

When packing for your math test, you remember that your school's ban on chewing gum has come into force today, so you derive *no* benefit from taking your gum to school: 

In [19]:
gum_ban = math_test.copy()
gum_ban.loc['chewing gum', 'reward'] = 0

<div class="alert alert-warning">
Using the methods we discussed above, can you create a new knapsack problem object to solve this <b>new</b> problem? How does the solution differ? from that which you found in <b>An Important Math Test</b>?
</div>

## Challenge: If I catch you without your shoes again one more time...!

Your Physical Education teacher is very upset that you have "forgotten" to bring your spare shoes so often.
<div class="alert alert-warning">
Can you implement a <i>new constraint</i> that makes sure you bring your spare shoes to school? If not, how high do you have to set the "spare shoes" reward until it becomes optimal to bring the shoes? 
</div>

# Review

Mathematical programs are a way of formalising our decision-making structures. They help us make decisions about complicated systems of equations and reason about how *good* our solutions are relative to other possible solutions. Every problem has three parts: 

1. the *decision variables* that represent the decisions we intend to make,
2. the *objective function* that represents how "good" a given solution to our problem is, and
3. the *constraints* that represent the limitations on decision variables that affect our solutions.

We formulate a mathematical programming using these three structures, and then *solve* the problem using a solution technique. *Mathematical programming* problems show up in a very large variety of settings, are computationally challenging to solve optimally, and are particularly useful in geography because they let us formalise our decision-making about spatial planning systems. We have solved an *aspatial knapsack* problem in order to illustrate the structure of general mathematical programming problems, and will use the concepts and vocabulary here to structure more complex *spatial* optimisation problem. 


In our next notebook, we will cover a family of problems known as the *location-allocation* family, when trying to solve a classically-useful problem: the *p*-median problem. 